# RL Coursework 1


**Date set:** Feb 26, 2018 

**Date due:** 8 pm on March 7, 2018 

**Total number of marks:** 10 

**What to submit:** Completed workbook (.ipynb file)

**Where to submit:** CM50270 Moodle page

The lab exercises in this workbook will determine 10% of your final mark for CM50270. 

You need to work on this coursework on your own. You are welcome to discuss it with your classmates but you must write your own code.  

Remember to save your work regularly.

You must comply with the universities plagiarism guidelines: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html


## Introduction


In this coursework, you will implement the Value Iteration algorithm to compute an optimal policy for three different (but related) Markov Decision Processes. For your reference, the pseudo-code for the algorithm is reproduced below from the textbook (Reinforcement Learning, Sutton & Barto, 1998). 

<img src="images/value_iteration.png" style="width: 800px;"/>

Please note the following about the pseudo-code: The set $\mathcal{S}$ contains all non-terminal states, whereas $\mathcal{S}^+$ is the set of all states (terminal and non-terminal). The reward $r = r(s, a, s')$ is the expected immediate reward on transition from state $s$ to the next state $s'$ under action $a$. 

<img src="images/bombs and gold numbers.png" style="width: 300px;" align="left" caption="Figure 1"/>

The three problems you will solve use variants of the gridworld environment shown on the left. You should be familiar with this environment from the lectures and from your previous lab exercise. The grid squares in the figure are numbered as shown. In all three problems, the following is true: 

**Actions available:** The agent has four possible actions in each grid square. These are _west_, _north_, _south_, and _east_. If the direction of movement is blocked by a wall (for example, if the agent executes action south at grid square 1), the agent remains in the same grid square. 

**Collecting gold:** On its first arrival at a grid square that contains gold, the agent collects the gold. In order to collect the gold, the agent needs to transition into the grid square (containing the gold) from a different grid square. 

**Hitting the bomb:** On arrival at a grid square that contains the bomb, the agent activates the bomb. 

** Terminal states:** The game terminates when all gold is collected or when the bomb is activated. In Exercises 1 and 2, you can define terminal states to be grid squares 18 and 23. In Exercise 3, you will need to define terminal state(s) differently.


### Instructions ###
Set parameter $\theta$ to 1 to the power of -10. You can express that as `1e-10` in Python. 

Set all initial state values $V(s)$ to zero.

Do not use discounting (that is, set $\gamma=1$).

Use the following reward function: $-1$ for each navigation action (including when the action results in hitting the wall), an additional $+10$ for collecting each piece of gold, and an additional $-10$ for activating the bomb. For example, the immediate reward for transitioning into a square with gold is $-1 + 10 = +9$. 


## Exercise 1: Deterministic environment (3 marks)

In this first exercise, the agent is able to move in the intended direction with certainty. For example, if it executes action _north_ in grid square 0, it will transition to grid square 5 with probability 1. In other words, we have a deterministic environment. 

Compute the optimal policy using Value Iteration. 

Your method needs to output two one-dimensional numpy arrays with names `policy` and `v`. Both arrays should have a length of 25, with the element at index i representing grid cell i. Both arrays should be callable in the "solution cell" below! 

The array `policy` should be an array of strings that specifies an optimal action at each grid location. Please use the abbreviations "n", "e", "s", and "w" for the four actions. As an example, policy at index 0 needs to give "n", if _north_ is an optimal action in cell 0. The policy for a terminal state can be any value (direction). If there are multiple optimal actions from a state, any optimal action will be considered as a correct answer. 

The array `v` should be an array of floats that contains the expected return at each grid square (that is, the state value).

In [44]:
import numpy as np


class Gridworld:

    def __init__(self):

        self.theta = 1e-10
        self.gamma = 1
        self.epsilon = 0
        self.num_rows = 5
        self.num_cols = 5
        self.total_cells = self.num_rows * self.num_cols

        # Actions available
        self.actions = ["n", "e", "s", "w"]
        self.num_actions = len(self.actions)
        
        # Default policy array
        self.policy = np.array(
            ["n", "w", "s", "w", "e", "n", "w", "s", "w", "e", "n", "w", "s", "w", "e", "n", "w", "s", "e", "e", "n",
             "w", "s", "e", "e"])
        
    def north(self, state):
        north_state = state + self.num_cols
        if north_state < self.total_cells:
            return north_state
        else:
            return state

    def east(self, state):
        east_state = state + 1
        if east_state % self.num_cols > 0:
            return east_state
        else:
            return state

    def south(self, state):
        south_state = state - self.num_cols
        if south_state >= 0:
            return south_state
        else:
            return state

    def west(self, state):
        west_state = state - 1
        if west_state % self.num_cols < self.num_cols - 1:
            return west_state
        else:
            return state

    def is_terminal(self, state):
        if state == 18:
            return True
        elif state == 23:
            return True
        else:
            return False
        
environment = Gridworld()

v = np.zeros(environment.total_cells)

# Default policy
policy = environment.policy
theta = environment.theta
gamma = environment.gamma
actions = environment.actions
alpha = 1
done = False

while not done:

    delta = 0

    for state in range(0, 25):

        if not done:

            terminal = environment.is_terminal(state)

            if not terminal:

                # Reward value and current value function of current state
                current_value = v[state]

                # Gets index of neighbouring states
                north_state = environment.north(state)
                east_state = environment.east(state)
                south_state = environment.south(state)
                west_state = environment.west(state)

                # Array used to check if any are terminal
                state_prime_array = np.array([north_state, east_state, south_state, west_state])
                value_list = []

                for x in range(0, 4):

                    value = 0

                    for y in range(0, 4):

                        reward = -1

                        state_prime = state_prime_array[y]

                        if x == y:
                            alpha = 1
                        else:
                            alpha = 0

                        if state_prime == 18:
                            reward += -10

                        if state_prime == 23:
                            reward += 10

                        value += alpha * (reward + (gamma * v[state_prime]))

                    value_list.append(value)
                v[state] = max(value_list)
                best_action = np.argmax(value_list)

                new_value = v[state]

                # Updates delta value
                delta = max(delta, abs(current_value - new_value))

                # Updates policy
                policy[state] = actions[best_action]

    if delta < theta:
        done = True

        
for i in range(0, len(policy)):
    print("State:", i, "\tValue:", v[i], "\tPolicy:", policy[i])

State: 0 	Value: 3.0 	Policy: n
State: 1 	Value: 4.0 	Policy: n
State: 2 	Value: 5.0 	Policy: n
State: 3 	Value: 4.0 	Policy: n
State: 4 	Value: 5.0 	Policy: n
State: 5 	Value: 4.0 	Policy: n
State: 6 	Value: 5.0 	Policy: n
State: 7 	Value: 6.0 	Policy: n
State: 8 	Value: 5.0 	Policy: n
State: 9 	Value: 6.0 	Policy: n
State: 10 	Value: 5.0 	Policy: n
State: 11 	Value: 6.0 	Policy: n
State: 12 	Value: 7.0 	Policy: n
State: 13 	Value: 6.0 	Policy: e
State: 14 	Value: 7.0 	Policy: n
State: 15 	Value: 6.0 	Policy: n
State: 16 	Value: 7.0 	Policy: n
State: 17 	Value: 8.0 	Policy: n
State: 18 	Value: 0.0 	Policy: e
State: 19 	Value: 8.0 	Policy: n
State: 20 	Value: 7.0 	Policy: e
State: 21 	Value: 8.0 	Policy: e
State: 22 	Value: 9.0 	Policy: e
State: 23 	Value: 0.0 	Policy: e
State: 24 	Value: 9.0 	Policy: w


In [ ]:
# Do not delete this cell. Your code for Exercise 1 is tested here.

## Exercise 2: Stochastic environment (4 marks)

In this second exercise, the agent is not always able to execute its actions as intended. With probability 0.8, it moves in the intended direction. With probability 0.2, it moves in a random direction. For example, from grid square 0, if the agent executes action _north_, with probability 0.8, the action will work as intended. But with probability 0.2, the agent's motor control system will move in a random direction (including north). For example, with probability 0.05, it will try to move west (where it will be blocked by the wall and hence remain in grid square 0). Notice that the total probability of moving to square 5 (as intended) is 0.8 + 0.05 = 0.85.
 
Compute the optimal policy using Value Iteration.

Your value iteration method should output two one-dimensional numpy arrays with names `policy` and `v`, as in Exercise 1.

In [45]:
import numpy as np


environment = Gridworld()

# V(s) table
v = np.zeros(environment.total_cells)

# Default policy
policy = environment.policy
theta = environment.theta
gamma = environment.gamma
actions = environment.actions
done = False

while not done:

    delta = 0

    for state in range(0, 25):

        if not done:

            terminal = environment.is_terminal(state)

            if not terminal:

                # Reward value and current value function of current state
                current_value = v[state]

                # Array of adjacent tile states
                state_prime_array = np.array([environment.north(state), environment.east(state), environment.south(state), environment.west(state)])
                value_list = []

                for x in range(0, 4):

                    value = 0

                    for y in range(0, 4):

                        reward = -1
                        state_prime = state_prime_array[y]

                        if x == y:
                            alpha = 0.85
                        else:
                            alpha = 0.05

                        if state_prime == 18:
                            reward += -10

                        if state_prime == 23:
                            reward += 10

                        # Adds the sum of the probability to reach chosen new state
                        value += alpha * (reward + (gamma * v[state_prime]))

                    value_list.append(value)

                v[state] = max(value_list)
                best_action = np.argmax(value_list)

                new_value = v[state]

                # Updates delta value
                delta = max(delta, abs(current_value - new_value))

                # Updates policy
                policy[state] = actions[best_action]

    if delta < theta:
        done = True

for i in range(0, len(policy)):
    print("State:", i, "\tValue: %0.10f" % v[i], "\tPolicy:", policy[i])


State: 0 	Value: 1.3597920787 	Policy: n
State: 1 	Value: 2.1973367201 	Policy: n
State: 2 	Value: 2.4287875130 	Policy: n
State: 3 	Value: 1.5727216147 	Policy: n
State: 4 	Value: 2.5520245101 	Policy: n
State: 5 	Value: 2.4869953351 	Policy: n
State: 6 	Value: 3.4094598877 	Policy: n
State: 7 	Value: 3.6692296713 	Policy: n
State: 8 	Value: 2.6412293327 	Policy: e
State: 9 	Value: 3.7861011511 	Policy: n
State: 10 	Value: 3.6755093765 	Policy: n
State: 11 	Value: 4.6962138840 	Policy: n
State: 12 	Value: 4.9944186290 	Policy: n
State: 13 	Value: 3.2189157994 	Policy: e
State: 14 	Value: 5.1025098840 	Policy: n
State: 15 	Value: 4.8618511138 	Policy: n
State: 16 	Value: 5.9908758698 	Policy: n
State: 17 	Value: 6.3708243073 	Policy: n
State: 18 	Value: 0.0000000000 	Policy: e
State: 19 	Value: 6.4672159320 	Policy: n
State: 20 	Value: 6.0416932891 	Policy: e
State: 21 	Value: 7.2875663583 	Policy: e
State: 22 	Value: 8.6135995087 	Policy: e
State: 23 	Value: 0.0000000000 	Policy: e
St

In [ ]:
# Do not delete this cell. Your code for Exercise 2 is tested here.

## Exercise 3: Deterministic environment with two pieces of gold (3 marks)

<img src="images/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/> In this third exercise, the environment is identical to the environment in Exercise 1 with the following exception: there is an additional piece of gold on grid square 12. Recall from earlier instructions that the terminal state is reached only when _all_ gold is collected or when the bomb is activated.

Compute the optimal policy using Value Iteration. 

Hint: You will need to change your state representation. 

Your method should output two one-dimensional numpy arrays with names `policy` and `v`, as in Exercises 1 and 2. These arrays should specify the expected return and an optimal policy at the corresponding grid sqaure **before any gold is collected or a bomb is activated.** 

In [46]:
import numpy as np


class Gridworld_2:

    def __init__(self):

        self.theta = 1e-10
        self.gamma = 1
        self.epsilon = 0
        self.num_rows = 5
        self.num_cols = 5
        self.total_cells = self.num_rows * self.num_cols

        # Actions available
        self.actions = ["n", "e", "s", "w"]
        self.num_actions = len(self.actions)

        # Default policy array
        self.policy = np.array(
            ["n", "w", "s", "w", "e", "n", "w", "s", "w", "e", "n", "w", "s", "w", "e", "n", "w", "s", "e", "e", "n",
             "w", "s", "e", "e", "n", "w", "s", "w", "e", "n", "w", "s", "w", "e", "n", "w", "s", "w", "e", "n", "w",
             "s", "e", "e", "n",
             "w", "s", "e", "e", "n", "w", "s", "w", "e", "n", "w", "s", "w", "e", "n", "w", "s", "w", "e", "n", "w",
             "s", "e", "e", "n",
             "w", "s", "e", "e"])

    def north(self, state):

        if 0 <= state <= 24:
            north_state = state + self.num_cols
            if north_state < self.total_cells:
                return north_state
            else:
                return state

        elif 25 <= state <= 49:
            north_state = state + self.num_cols
            if north_state < self.total_cells * 2:
                return north_state
            else:
                return state
        else:
            north_state = state + self.num_cols
            if north_state < self.total_cells * 3:
                return north_state
            else:
                return state

    def east(self, state):

        east_state = state + 1
        if east_state % self.num_cols > 0:
            return east_state
        else:
            return state

    def south(self, state):

        if 0 <= state <= 24:
            south_state = state - self.num_cols
            if south_state >= 0:
                return south_state
            else:
                return state

        elif 25 <= state <= 49:
            south_state = state - self.num_cols
            if south_state >= 25:
                return south_state
            else:
                return state

        else:
            south_state = state - self.num_cols
            if south_state >= 50:
                return south_state
            else:
                return state

    def west(self, state):
        west_state = state - 1
        if west_state % self.num_cols < self.num_cols - 1:
            return west_state
        else:
            return state

    def is_terminal(self, state):

        if state == 18:
            return True
        elif state == 37:
            return True
        elif state == 43:
            return True
        elif state == 68:
            return True
        elif state == 73:
            return True
        else:
            return False

environment = Gridworld_2()

# V(s) tables
v = np.zeros(environment.total_cells * 3)

# Default policy
policy = environment.policy
theta = environment.theta
gamma = environment.gamma
actions = environment.actions
done = False

while not done:

    delta = 0

    for state in range(0, 75):

        if state == 12:
            state = state + 50
            special_state = True
        elif state == 23:
            state = state + 25
            special_state = True
        else:
            special_state = False

        # print("State: ", state)

        if not done:

            terminal = environment.is_terminal(state)

            if not terminal:

                # Reward value and current value function of current state
                current_value = v[state]
                # print("current value: ", current_value)

                # Array of adjacent tile states
                state_prime_array = np.array(
                    [environment.north(state), environment.east(state), environment.south(state),
                     environment.west(state)])
                # print("State: " + str(state) + " Moving states: " + str(state_prime_array))

                # Empty list to record state values
                value_list = []

                for x in range(0, 4):

                    value = 0

                    for y in range(0, 4):

                        reward = -1
                        state_prime = state_prime_array[y]

                        if x == y:
                            alpha = 1
                        else:
                            alpha = 0

                        # Adjust reward for Bomb States
                        if state_prime == 18 or state_prime == 43 or state_prime == 68:
                            reward += -10

                        if state_prime == 12 or state_prime == 23 or state_prime == 37 or state_prime == 73:
                            reward += 10

                        # Adds the sum of the probability to reach chosen new state
                        value += alpha * (reward + (gamma * v[state_prime]))

                    value_list.append(value)

                if special_state:

                    if state == 62:
                        v[state - 50] = max(value_list)
                        best_action = np.argmax(value_list)
                        new_value = v[state - 50]
                        policy[state - 50] = actions[best_action]
                    else:
                        v[state - 25] = max(value_list)
                        best_action = np.argmax(value_list)
                        new_value = v[state - 25]
                        policy[state - 25] = actions[best_action]
                else:
                    v[state] = max(value_list)
                    best_action = np.argmax(value_list)
                    new_value = v[state]
                    policy[state] = actions[best_action]

                # Updates delta value
                delta = max(delta, abs(current_value - new_value))
    
    if delta < theta:
        done = True


policy = policy[:25]
v = v[:25]
        
for i in range(0, len(policy)):
    print("State:", i, "\tValue: %0.10f" % v[i], "\tPolicy:", policy[i])

State: 0 	Value: 13.0000000000 	Policy: n
State: 1 	Value: 14.0000000000 	Policy: n
State: 2 	Value: 15.0000000000 	Policy: n
State: 3 	Value: 14.0000000000 	Policy: n
State: 4 	Value: 13.0000000000 	Policy: n
State: 5 	Value: 14.0000000000 	Policy: n
State: 6 	Value: 15.0000000000 	Policy: n
State: 7 	Value: 16.0000000000 	Policy: n
State: 8 	Value: 15.0000000000 	Policy: n
State: 9 	Value: 14.0000000000 	Policy: n
State: 10 	Value: 15.0000000000 	Policy: e
State: 11 	Value: 16.0000000000 	Policy: e
State: 12 	Value: 7.0000000000 	Policy: n
State: 13 	Value: 16.0000000000 	Policy: w
State: 14 	Value: 15.0000000000 	Policy: w
State: 15 	Value: 14.0000000000 	Policy: e
State: 16 	Value: 15.0000000000 	Policy: e
State: 17 	Value: 16.0000000000 	Policy: s
State: 18 	Value: 0.0000000000 	Policy: e
State: 19 	Value: 15.0000000000 	Policy: n
State: 20 	Value: 14.0000000000 	Policy: e
State: 21 	Value: 15.0000000000 	Policy: e
State: 22 	Value: 16.0000000000 	Policy: e
State: 23 	Value: 7.000

In [40]:
# Do not delete this cell. Your code for Exercise 3 is tested here.